### Remax

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from splinter import Browser
import warnings
warnings.filterwarnings('ignore')
print('Libraries imported!')

Libraries imported!


In [2]:
house_address = []
house_details = []

base_url = 'https://www.remax.ca/ab/calgary-real-estate?page='
urls = [base_url + str(x) for x in range(1,2)]

for url in urls:
    
    #browser.visit(url)
    time.sleep(5)

    # Parse HTML with Beautiful Soup
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        addresses = soup.find_all('div', class_='left-content flex-one')
        for address in addresses:
            house_address.append(address.text)
    except:
        house_address.append('None')
        
    try:
        details = soup.find_all('div', class_='property-details')
        for detail in details:
            house_details.append(detail.text)
    except:
        house_details.append('None')

In [5]:
house_address[:5]

[' $329,900 4312 49 ST NE, Calgary, AB, T1Y 2Z3',
 ' $464,900 3916 VANCOUVER CRES NW, Calgary, AB, T3A 0M1',
 ' $119,800 102 - 99 ARBOUR LAKE RD NW, Calgary, AB, T3G 4E4',
 ' $614,900 63 CRANARCH CIR SE, Calgary, AB, T3M 0S3',
 ' $329,900 32 COPPERPOND Close SE, Calgary, AB, T2Z 0Y9']

In [6]:
house_details[:5]

['4 bed|1 + 1 bath|122 sqft|house',
 '4 bed|2 bath|1043 sqft|house',
 '3 bed|1 + 1 bath|1316 sqft',
 '4 bed|3 + 1 bath|2210 sqft|house',
 '3 bed|2 + 1 bath|1323 sqft|townhouse']

In [3]:
address_df = pd.DataFrame(house_address)

new_df = address_df[0].str.split(' ', 2, expand=True)
new_df["price"] = new_df[1].str.replace("$", "")
new_df["price"] = new_df["price"].str.replace(",", "")

del new_df[0]
del new_df[1]
new_df.head()

,2,price
0,"4312 49 ST NE, Calgary, AB, T1Y 2Z3",329900
1,"3916 VANCOUVER CRES NW, Calgary, AB, T3A 0M1",464900
2,"102 - 99 ARBOUR LAKE RD NW, Calgary, AB, T3G 4E4",119800
3,"63 CRANARCH CIR SE, Calgary, AB, T3M 0S3",614900
4,"32 COPPERPOND Close SE, Calgary, AB, T2Z 0Y9",329900


In [22]:
final_df = new_df[2].str.split(', Calgary, AB, ', expand=True)
final_df.head()

,0,1
0,9803 ELBOW DR SW,T2V 1M4
1,140 CITADEL CREST CIR NW,T3G 4G3
2,275 TARAVISTA ST NE,T3J 4S5
3,25 DOUGLASBANK RISE SE,T2Z 2C5
4,79 GALBRAITH DR SW,T3E 4Z5


In [23]:
df_add = pd.concat([new_df, final_df], axis=1)
del df_add[2]
df_add.columns = ["price", "address", "postal_code"]
df_add.head()

,price,address,postal_code
0,489900,9803 ELBOW DR SW,T2V 1M4
1,489000,140 CITADEL CREST CIR NW,T3G 4G3
2,449999,275 TARAVISTA ST NE,T3J 4S5
3,469900,25 DOUGLASBANK RISE SE,T2Z 2C5
4,351999,79 GALBRAITH DR SW,T3E 4Z5


In [24]:
details = pd.DataFrame(house_details)

details_df = details[0].str.split('|', expand=True)
details_df

del details_df[2]

details_df.columns = ["bedrooms", "bath", "property_type"]
details_df.head()

,bedrooms,bath,property_type
0,4 bed,2 bath,house
1,3 bed,2 + 1 bath,house
2,5 bed,2 + 1 bath,house
3,4 bed,2 + 1 bath,house
4,N/A bed,N/A bath,house


In [25]:
calgary_df = pd.concat([df_add, details_df], axis=1)
calgary_df.head()

,price,address,postal_code,bedrooms,bath,property_type
0,489900,9803 ELBOW DR SW,T2V 1M4,4 bed,2 bath,house
1,489000,140 CITADEL CREST CIR NW,T3G 4G3,3 bed,2 + 1 bath,house
2,449999,275 TARAVISTA ST NE,T3J 4S5,5 bed,2 + 1 bath,house
3,469900,25 DOUGLASBANK RISE SE,T2Z 2C5,4 bed,2 + 1 bath,house
4,351999,79 GALBRAITH DR SW,T3E 4Z5,N/A bed,N/A bath,house


In [26]:
calgary_df.to_csv('calgary_df.csv', encoding="utf-8")

----------------

### Walk Score

In [27]:
post_code_list = []

for i in calgary_df["postal_code"]:
    post_code_list.append(i)

In [28]:
scores_walk = []
scores_bike = []
scores_transit = []

for i in post_code_list:
    postal_code = i.replace(" ", "%20")
    url_score = "https://www.walkscore.com/score/" + str(postal_code)

    time.sleep(2)

    # Parse HTML with Beautiful Soup
    response = requests.get(url_score)
    code_soup = BeautifulSoup(response.text, 'html.parser')

    try:
        if 'pp.walk.sc/badge/walk/score' in str(code_soup):
            ws = str(code_soup).split('pp.walk.sc/badge/walk/score/')[1][:2].replace('.','')
            scores_walk.append(ws)
        else:
            ws = 'N/A'
            scores_walk.append(ws)
        if 'pp.walk.sc/badge/bike/score' in str(code_soup):
            bs = str(code_soup).split('pp.walk.sc/badge/bike/score/')[1][:2].replace('.','')
            scores_bike.append(bs)
        else:
            bs = 'N/A'
            scores_bike.append(bs)
        if 'pp.walk.sc/badge/transit/score' in str(code_soup):
            ts = str(code_soup).split('pp.walk.sc/badge/transit/score/')[1][:2].replace('.','')
            scores_transit.append(ts)
        else:
            ts = 'N/A'
            scores_transit.append(ts)
    except:
        ws = 'N/A'
        scores_walk.append(ws)
        bs = 'N/A'
        scores_bike.append(bs)
        ts = 'N/A'
        scores_transit.append(ts)

In [29]:
score_df_trans = {'postal_code':post_code_list, 'walk_score':scores_walk, 'bike_score':scores_bike, 'transit_score':scores_transit}
score_df = pd.DataFrame(score_df_trans)
score_df.head()

,postal_code,walk_score,bike_score,transit_score
0,T2V 1M4,58,61,55
1,T3G 4G3,23,65,38
2,T3J 4S5,10,62,49
3,T2Z 2C5,22,65,41
4,T3E 4Z5,43,67,47
